# setup

In [4]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
!pip install tensorflow_addons
import tensorflow_addons as tfa
from tensorflow import keras
print("TF version: ", tf.__version__)
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")    

from tensorflow.python.keras.utils.data_utils import Sequence

sklearn version:  0.23.1
Defaulting to user installation because normal site-packages is not writeable
TF version:  2.6.0
GPU installed:  True
2 Physical GPUs, 2 Logical GPUs


# Get the data

In [5]:
#작성자 전민재, hsh
import csv
def load_Emist(exsitNumpy=False, needTranspose=True):
    if (exsitNumpy == False):
        #"./emnist-byclass-test.csv"
        #train-set
        #"공용/datasets/Emnist/" 위치에 csv 저장 
        csv_train_data_file = open("../공용/datasets/Emnist/train_data.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
        csv_train_label_file = open("../공용/datasets/Emnist/train_label.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
    
        
        f_train = csv.reader(csv_train_data_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
        l_train=csv.reader(csv_train_label_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
        #valid-set
    
        csv_valid_data_file = open("../공용/datasets/Emnist/valid_data.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
        csv_valid_label_file = open("../공용/datasets/Emnist/valid_label.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
        
        f_valid = csv.reader(csv_valid_data_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
        l_valid = csv.reader(csv_valid_label_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
        X_train=[]
        y_train=[]
        X_valid=[]
        y_valid=[]
        for i, row in enumerate(f_train):
            #행마다 int로 형변환
            for idx, char in enumerate(row):
                row[idx]=int(char)
            #train
            #data 추가    
            X_train.append(row)
            
        for i, row in enumerate(f_valid):
            #행마다 int로 형변환
            for idx, char in enumerate(row):
                row[idx]=int(char)
            #valid
            #data 추가    
            X_valid.append(row)


        for i, row in enumerate(l_train):

            #train
            #label 추가
            y_train.append(int(row[0]))
            
        for i, row in enumerate(l_valid):

            #valid
            #label 추가
            y_valid.append(int(row[0]))
            
            
        

        #train
        X_train=np.array(X_train,dtype=np.uint8)
        X_train=X_train.reshape(-1,28,28)
        #valid
        X_valid=np.array(X_valid,dtype=np.uint8)
        X_valid=X_valid.reshape(-1,28,28)
        
        csv_train_data_file.close()
        csv_train_label_file.close()
        csv_valid_data_file.close()
        csv_valid_label_file.close()
        
        #kaggle dataset이 시계반대방향으로 90도 회전 되있고 상하 반전 되어있음
        def rotate_90(m):
            N = len(m)
            ret = [[0] * N for _ in range(N)]

            for r in range(N):
                for c in range(N):
                    ret[c][N-1-r] = m[r][c]
            return np.array(ret,dtype=np.uint8)

        def vreflect(m):
            N = len(m)
            ret = [[0] * N for _ in range(N)]

            for r in range(N):
                for c in range(N):
                    ret[r][c] = m[r][N-1-c]
            return np.array(ret,dtype=np.uint8)
        
        if needTranspose == True:
            #train
            #회전
            for idx,i in enumerate(X_train):
                X_train[idx]=rotate_90(i)
            #상하반전
            for idx,i in enumerate(X_train):
                X_train[idx]=vreflect(i)

            #valid
            #회전
            for idx,i in enumerate(X_valid):
                X_valid[idx]=rotate_90(i)
            #상하반전
            for idx,i in enumerate(X_valid):
                X_valid[idx]=vreflect(i)

       
        
        
        #저장용
        #train
        #(533917,28,28)
        
        np.save('../공용/X_train',np.array(X_train,dtype=np.uint8))
        np.save('../공용/y_train',np.array(y_train,dtype=np.uint8))
        #valid
        np.save('../공용/X_valid',np.array(X_valid,dtype=np.uint8))
        np.save('../공용/y_valid',np.array(y_valid,dtype=np.uint8))
        
   
    X_train=np.load('../공용/X_train.npy')
    y_train=np.load('../공용/y_train.npy')
    X_valid=np.load('../공용/X_valid.npy')
    y_valid=np.load('../공용/y_valid.npy')
    
    return X_train, y_train, X_valid, y_valid


In [6]:
#황성현
#npy 파일이 존재할 경우 exsitNumpy = True, 업으면 False
X_train, y_train, X_valid, y_valid = load_Emist(exsitNumpy=True)
X_train.shape,X_train.dtype

((533917, 28, 28), dtype('uint8'))

In [7]:
#황성현
X_train=tf.reshape(X_train,[-1,28,28,1])
X_valid=tf.reshape(X_valid,[-1,28,28,1])

In [8]:
#황성현,jmj
X_train.shape,X_valid.shape,X_train.dtype

(TensorShape([533917, 28, 28, 1]), TensorShape([164015, 28, 28, 1]), tf.uint8)

In [9]:
#황성현, jmj
X_train_resize=np.uint8(tf.image.resize(X_train, [32, 32]))
X_valid_resize=np.uint8(tf.image.resize(X_valid, [32, 32]))


In [10]:
#황성현
X_train_resize.shape,X_valid_resize.dtype,X_valid_resize.shape

((533917, 32, 32, 1), dtype('uint8'), (164015, 32, 32, 1))

# train the model

# Inverted Rsidual Block

In [11]:
#황성현
class Inveted_Residual_Block(keras.layers.Layer):
    def __init__(self,c,s=1,t=1,n=1,**kwargs):
        super().__init__(**kwargs)
        self.activation = keras.layers.ReLU(max_value=6)
        self.main_layers = [
            
            ]
        self.strides_layers=[
           ]
        self.s=s
        self.t=t
        self.c=c
        self.n=n
        
    def build(self,batch_input_shape):
        print(self.name,": batch_input_shape =",batch_input_shape)
        
        self.main_layers = [
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=batch_input_shape[-1]*self.t, kernel_size=1,strides=1, 
                                padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3, strides=1,padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=self.c,kernel_size=1, strides=1,padding="SAME", activation=self.activation),
            ]
        self.strides_layers=[
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=batch_input_shape[-1]*self.t, kernel_size=1,strides=1,
                                padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3, strides=self.s,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=self.c, kernel_size=1,strides=1,padding="SAME",activation=self.activation),
        ]

    def call(self, inputs):
        Z = inputs
        
        for n in range(0,self.n):
            if n == 0:
                for layer in self.strides_layers:
                    Z = layer(Z)
            else:
                skip_Z = Z
                for layer in self.main_layers:
                    Z = layer(Z)
                
                Z += skip_Z
    
        return Z
    
    def get_config(self):
        base=super().get_config()
        return{**base,"s":self.s,"t":self.t,"c":self.c,"n":self.n}

# SMV2L

In [12]:
#황성현
#multygpu
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    n_classes=62
    smv2_l=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_l.summary()
    smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer="nadam",metrics=["accuracy"])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
IRB1 : batc

In [13]:
#황성현
from datetime import datetime
batch_size=32
logs = "logs/" + datetime.now().strftime("SMV2L_NP%Y%m%d-%H%M%S")
print(logs)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_nadam_np_best.h5",save_best_only=True)
check_last_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_nadam_np_last.h5",save_best_only=False)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
lr_callback=keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.5)
history=smv2_l.fit(X_train_resize,y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize,y_valid),
                    epochs=300,callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb,lr_callback])

logs/SMV2L_NP20220531-231253
Epoch 1/300
INFO:tensorflow:batch_all_reduce: 116 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 116 all-reduces with algorithm = nccl, num_packs = 1
16685/16685 [==============================] - 599s 34ms/step - loss: 0.7854 - accuracy: 0.7541 - val_loss: 0.4821 - val_accuracy: 0.8337
Epoch 2/300
16685/16685 [==============================] - 561s 34ms/step - loss: 0.4899 - accuracy: 0.8294 - val_loss: 0.4498 - val_accuracy: 0.8405
Epoch 3/300
16685/16685 [==============================] - 560s 34ms/step - loss: 0.4555 - accuracy: 0.8395 - val_loss: 0.4345 - val_accuracy: 0.8404
Epoch 4/300
16685/16685 [==============================] - 561s 34ms/step - loss: 0.4398 - accuracy: 0.8438 - val_loss: 0.4139 - val_accuracy: 0.8516
Epoch 5/300
16685/16685 [==============================] - 560s 34ms/step - loss: 0.4301 - accuracy: 0.8463 - val_loss: 0.4139 - val_accuracy: 0.8514
Epoch 6/300
16685/16685 [======================

# SMv2P

In [14]:
#황성현
#multygpu
with strategy.scope():
    n_classes=62
    smv2_p=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_p.summary()
    smv2_p.compile(loss="sparse_categorical_crossentropy",optimizer="nadam",metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 32)
IRB4 : batch_input_shape = (None, 4, 4, 64)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 32, 32, 1)         4         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        320       
_________________________________________________________________
IRB1 (Inveted_Residual_Block (None, 16, 16, 16)        2288      
_________________________________________________________________
IRB2 (Inveted_Residual_Block (None, 8, 8, 32)          9856      
_________________________________________________________________
IRB3 (Inveted_Residual_Block (None, 4, 4, 64)    

In [15]:
#황성현
from datetime import datetime
batch_size=32
logs = "logs/" + datetime.now().strftime("SMV2P_NP%Y%m%d-%H%M%S")
print(logs)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2p_nadam_np_best.h5",save_best_only=True)
check_last_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2p_nadam_np_last.h5",save_best_only=False)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
lr_callback=keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.5)
history=smv2_p.fit(X_train_resize,y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize,y_valid),
                    epochs=300,callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb,lr_callback])

logs/SMV2P_NP20220601-065122
Epoch 1/300
INFO:tensorflow:batch_all_reduce: 116 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 116 all-reduces with algorithm = nccl, num_packs = 1
16685/16685 [==============================] - 592s 34ms/step - loss: 0.7944 - accuracy: 0.7514 - val_loss: 0.5066 - val_accuracy: 0.8272
Epoch 2/300
16685/16685 [==============================] - 556s 33ms/step - loss: 0.4889 - accuracy: 0.8293 - val_loss: 0.4365 - val_accuracy: 0.8453
Epoch 3/300
16685/16685 [==============================] - 555s 33ms/step - loss: 0.4538 - accuracy: 0.8393 - val_loss: 0.4189 - val_accuracy: 0.8495
Epoch 4/300
16685/16685 [==============================] - 556s 33ms/step - loss: 0.4384 - accuracy: 0.8438 - val_loss: 0.4072 - val_accuracy: 0.8520
Epoch 5/300
16685/16685 [==============================] - 557s 33ms/step - loss: 0.4275 - accuracy: 0.8469 - val_loss: 0.4084 - val_accuracy: 0.8534
Epoch 6/300
16685/16685 [======================

# fine tuning

# SMV2L

adam

In [16]:
#황성현
#multygpu
with strategy.scope():
    n_classes=62
    smv2_l=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_l.summary()
    smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 32)
IRB4 : batch_input_shape = (None, 4, 4, 64)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 32, 32, 1)         4         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        320       
_________________________________________________________________
IRB1 (Inveted_Residual_Block (None, 16, 16, 16)        2288      
_________________________________________________________________
IRB2 (Inveted_Residual_Block (None, 8, 8, 32)          9856      
_________________________________________________________________
IRB3 (Inveted_Residual_Block (None, 4, 4, 64)    

In [17]:
#황성현
from datetime import datetime
batch_size=32
logs = "logs/" + datetime.now().strftime("SMV2L_NP_ADAM%Y%m%d-%H%M%S")
print(logs)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_adam_np_best.h5",save_best_only=True)
check_last_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_adam_np_last.h5",save_best_only=False)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
lr_callback=keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.4)
history=smv2_l.fit(X_train_resize,y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize,y_valid),
                    epochs=300,callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb,lr_callback])

logs/SMV2L_NP_ADAM20220601-144146
Epoch 1/300
INFO:tensorflow:batch_all_reduce: 116 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 116 all-reduces with algorithm = nccl, num_packs = 1
16685/16685 [==============================] - 475s 28ms/step - loss: 0.7943 - accuracy: 0.7527 - val_loss: 0.4928 - val_accuracy: 0.8302
Epoch 2/300
16685/16685 [==============================] - 452s 27ms/step - loss: 0.4894 - accuracy: 0.8294 - val_loss: 0.4493 - val_accuracy: 0.8399
Epoch 3/300
16685/16685 [==============================] - 453s 27ms/step - loss: 0.4544 - accuracy: 0.8391 - val_loss: 0.4158 - val_accuracy: 0.8479
Epoch 4/300
16685/16685 [==============================] - 454s 27ms/step - loss: 0.4386 - accuracy: 0.8439 - val_loss: 0.4076 - val_accuracy: 0.8511
Epoch 5/300
16685/16685 [==============================] - 452s 27ms/step - loss: 0.4270 - accuracy: 0.8471 - val_loss: 0.4151 - val_accuracy: 0.8505
Epoch 6/300
16685/16685 [=================

Epoch 54/300
16685/16685 [==============================] - 450s 27ms/step - loss: 0.2861 - accuracy: 0.8886 - val_loss: 0.3435 - val_accuracy: 0.8733
Epoch 55/300
16685/16685 [==============================] - 450s 27ms/step - loss: 0.2858 - accuracy: 0.8890 - val_loss: 0.3444 - val_accuracy: 0.8735
Epoch 56/300
16685/16685 [==============================] - 450s 27ms/step - loss: 0.2854 - accuracy: 0.8889 - val_loss: 0.3441 - val_accuracy: 0.8735


rmsprop

In [18]:
#황성현
#multygpu
with strategy.scope():
    n_classes=62
    smv2_l=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_l.summary()
    smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer=keras.optimizers.RMSprop(),metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 32)
IRB4 : batch_input_shape = (None, 4, 4, 64)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_6 (Batch (None, 32, 32, 1)         4         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 32)        320       
_________________________________________________________________
IRB1 (Inveted_Residual_Block (None, 16, 16, 16)        2288      
_________________________________________________________________
IRB2 (Inveted_Residual_Block (None, 8, 8, 32)          9856      
_________________________________________________________________
IRB3 (Inveted_Residual_Block (None, 4, 4, 64)    

In [19]:
#황성현
from datetime import datetime
batch_size=32
logs = "logs/" + datetime.now().strftime("SMV2L_NP_RMSPROP%Y%m%d-%H%M%S")
print(logs)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_rmsprop_np_best.h5",save_best_only=True)
check_last_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_rmsprop_np_last.h5",save_best_only=False)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
lr_callback=keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.4)
history=smv2_l.fit(X_train_resize,y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize,y_valid),
                    epochs=300,callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb,lr_callback])

logs/SMV2L_NP_RMSPROP20220601-214621
Epoch 1/300
INFO:tensorflow:batch_all_reduce: 116 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 116 all-reduces with algorithm = nccl, num_packs = 1
16685/16685 [==============================] - 518s 30ms/step - loss: 0.7920 - accuracy: 0.7606 - val_loss: 0.5327 - val_accuracy: 0.8200
Epoch 2/300
16685/16685 [==============================] - 491s 29ms/step - loss: 0.5483 - accuracy: 0.8217 - val_loss: 0.4953 - val_accuracy: 0.8329
Epoch 3/300
16685/16685 [==============================] - 492s 29ms/step - loss: 0.5209 - accuracy: 0.8280 - val_loss: 0.4872 - val_accuracy: 0.8209
Epoch 4/300
16685/16685 [==============================] - 492s 30ms/step - loss: 0.5073 - accuracy: 0.8324 - val_loss: 0.4669 - val_accuracy: 0.8432
Epoch 5/300
16685/16685 [==============================] - 493s 30ms/step - loss: 0.5054 - accuracy: 0.8327 - val_loss: 0.4616 - val_accuracy: 0.8433
Epoch 6/300
16685/16685 [==============

neg

In [20]:
#황성현
#multygpu
with strategy.scope():
    n_classes=62
    smv2_l=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_l.summary()
    smv2_l.compile(loss="sparse_categorical_crossentropy",
                   optimizer=keras.optimizers.SGD(learning_rate=0.01,momentum=0.9,nesterov=True),
                   metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 32)
IRB4 : batch_input_shape = (None, 4, 4, 64)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_8 (Batch (None, 32, 32, 1)         4         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 32)        320       
_________________________________________________________________
IRB1 (Inveted_Residual_Block (None, 16, 16, 16)        2288      
_________________________________________________________________
IRB2 (Inveted_Residual_Block (None, 8, 8, 32)          9856      
_________________________________________________________________
IRB3 (Inveted_Residual_Block (None, 4, 4, 64)    

In [21]:
#황성현
from datetime import datetime
batch_size=32
logs = "logs/" + datetime.now().strftime("SMV2L_NP_NAG%Y%m%d-%H%M%S")
print(logs)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_nag_np_best.h5",save_best_only=True)
check_last_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_nag_np_last.h5",save_best_only=False)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
lr_callback=keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.4)
history=smv2_l.fit(X_train_resize,y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize,y_valid),
                    epochs=300,callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb,lr_callback])

logs/SMV2L_NP_NAG20220602-043120
Epoch 1/300
INFO:tensorflow:batch_all_reduce: 116 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 116 all-reduces with algorithm = nccl, num_packs = 1
16685/16685 [==============================] - 461s 27ms/step - loss: 0.8030 - accuracy: 0.7519 - val_loss: 0.4969 - val_accuracy: 0.8256
Epoch 2/300
16685/16685 [==============================] - 440s 26ms/step - loss: 0.5001 - accuracy: 0.8251 - val_loss: 0.4486 - val_accuracy: 0.8437
Epoch 3/300
16685/16685 [==============================] - 441s 26ms/step - loss: 0.4502 - accuracy: 0.8394 - val_loss: 0.4200 - val_accuracy: 0.8468
Epoch 4/300
16685/16685 [==============================] - 440s 26ms/step - loss: 0.4248 - accuracy: 0.8466 - val_loss: 0.4087 - val_accuracy: 0.8515
Epoch 5/300
16685/16685 [==============================] - 440s 26ms/step - loss: 0.4087 - accuracy: 0.8511 - val_loss: 0.3946 - val_accuracy: 0.8582
Epoch 6/300
16685/16685 [==================

momentum

In [22]:
#황성현
#multygpu
with strategy.scope():
    n_classes=62
    smv2_l=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_l.summary()
    smv2_l.compile(loss="sparse_categorical_crossentropy",
                   optimizer=keras.optimizers.SGD(learning_rate=0.01,momentum=0.9),
                   metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 32)
IRB4 : batch_input_shape = (None, 4, 4, 64)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_10 (Batc (None, 32, 32, 1)         4         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 32)        320       
_________________________________________________________________
IRB1 (Inveted_Residual_Block (None, 16, 16, 16)        2288      
_________________________________________________________________
IRB2 (Inveted_Residual_Block (None, 8, 8, 32)          9856      
_________________________________________________________________
IRB3 (Inveted_Residual_Block (None, 4, 4, 64)    

In [ ]:
#황성현
from datetime import datetime
batch_size=32
logs = "logs/" + datetime.now().strftime("SMV2L_NP_MOMENTUM%Y%m%d-%H%M%S")
print(logs)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_momentum_np_best.h5",save_best_only=True)
check_last_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_momentum_np_last.h5",save_best_only=False)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
lr_callback=keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.4)
history=smv2_l.fit(X_train_resize,y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize,y_valid),
                    epochs=300,callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb,lr_callback])

logs/SMV2L_NP_MOMENTUM20220602-094148
Epoch 1/300
16685/16685 [==============================] - 461s 27ms/step - loss: 0.7607 - accuracy: 0.7621 - val_loss: 0.4881 - val_accuracy: 0.8286
Epoch 2/300
16685/16685 [==============================] - 441s 26ms/step - loss: 0.4912 - accuracy: 0.8280 - val_loss: 0.4478 - val_accuracy: 0.8427
Epoch 3/300
16685/16685 [==============================] - 440s 26ms/step - loss: 0.4425 - accuracy: 0.8418 - val_loss: 0.4125 - val_accuracy: 0.8496
Epoch 4/300
16685/16685 [==============================] - 441s 26ms/step - loss: 0.4173 - accuracy: 0.8491 - val_loss: 0.4034 - val_accuracy: 0.8538
Epoch 5/300
16685/16685 [==============================] - 440s 26ms/step - loss: 0.4006 - accuracy: 0.8535 - val_loss: 0.3935 - val_accuracy: 0.8574
Epoch 6/300
16685/16685 [==============================] - 440s 26ms/step - loss: 0.3868 - accuracy: 0.8572 - val_loss: 0.3890 - val_accuracy: 0.8583
Epoch 7/300
16685/16685 [==============================] - 439

# SMV2P

adam

In [ ]:
#황성현
#multygpu
with strategy.scope():
    n_classes=62
    smv2_p=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_p.summary()
    smv2_p.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
#황성현
from datetime import datetime
batch_size=32
logs = "logs/" + datetime.now().strftime("SMV2P_NP_ADAM%Y%m%d-%H%M%S")
print(logs)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2p_nadam_np_best.h5",save_best_only=True)
check_last_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2p_nadam_np_last.h5",save_best_only=False)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
lr_callback=keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.4)
history=smv2_p.fit(X_train_resize,y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize,y_valid),
                    epochs=300,callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb,lr_callback])

rmsprop

In [ ]:
#황성현
#multygpu
with strategy.scope():
    n_classes=62
    smv2_p=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_p.summary()
    smv2_p.compile(loss="sparse_categorical_crossentropy",optimizer="rmsprop",metrics=["accuracy"])

In [ ]:
#황성현
from datetime import datetime
batch_size=32
logs = "logs/" + datetime.now().strftime("SMV2P_NP_RMSPROP%Y%m%d-%H%M%S")
print(logs)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2p_rmsprop_np_best.h5",save_best_only=True)
check_last_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2p_rmsprop_np_last.h5",save_best_only=False)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
lr_callback=keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.4)
history=smv2_p.fit(X_train_resize,y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize,y_valid),
                    epochs=300,callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb,lr_callback])

neg

In [ ]:
#황성현
#multygpu
with strategy.scope():
    n_classes=62
    smv2_p=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_p.summary()
    smv2_p.compile(loss="sparse_categorical_crossentropy",
                   optimizer=keras.optimizers.SGD(learning_rate=0.01,momentum=0.9,nesterov=True)
                   ,metrics=["accuracy"])

In [ ]:
#황성현
from datetime import datetime
batch_size=32
logs = "logs/" + datetime.now().strftime("SMV2P_NP_NAG%Y%m%d-%H%M%S")
print(logs)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2p_nag_np_best.h5",save_best_only=True)
check_last_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2p_nag_np_last.h5",save_best_only=False)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
lr_callback=keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.4)
history=smv2_p.fit(X_train_resize,y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize,y_valid),
                    epochs=300,callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb,lr_callback])

Momentum

In [ ]:
#황성현
#multygpu
with strategy.scope():
    n_classes=62
    smv2_p=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=4,c=32,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_p.summary()
    smv2_p.compile(loss="sparse_categorical_crossentropy",
                   optimizer=keras.optimizers.SGD(learning_rate=0.01,momentum=0.9)
                   ,metrics=["accuracy"])

In [ ]:
#황성현
from datetime import datetime
batch_size=32
logs = "logs/" + datetime.now().strftime("SMV2P_NP_MOMENTUM%Y%m%d-%H%M%S")
print(logs)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2p_momentum_np_best.h5",save_best_only=True)
check_last_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2p_momentum_np_last.h5",save_best_only=False)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
lr_callback=keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.4)
history=smv2_p.fit(X_train_resize,y_train, batch_size = batch_size,
                    validation_data=(X_valid_resize,y_valid),
                    epochs=300,callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb,lr_callback])